In [1]:
import pandas as pd
import pandas.api.types as ptypes
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sqlalchemy import create_engine, text
import sys

### Import data from local postgres database

In [2]:
# --- 1. Database Configuration ---
# Store configuration in a dictionary for clarity.
# IMPORTANT: Replace placeholder values with your actual credentials.
# Consider using environment variables or a config file for production/shared code.
DB_CONFIG = {
    "type": "postgresql",
    "driver": "psycopg2",
    "user": "postgres", 
    "pass": "password", 
    "host": "localhost", 
    "port": "5432", 
    "name": "contoso_100k" 
}

# --- 2. Database Connection Function ---
def create_db_engine(config):
    """
    Creates a SQLAlchemy engine for the database connection.

    Args:
        config (dict): Dictionary containing database connection parameters.

    Returns:
        sqlalchemy.engine.Engine or None: The created engine, or None if connection fails.
    """
    try:
        # Construct the database connection URL
        connection_url = (
            f"{config['type']}+{config['driver']}://"
            f"{config['user']}:{config['pass']}@"
            f"{config['host']}:{config['port']}/{config['name']}"
        )
        engine = create_engine(connection_url)
        # Test connection briefly
        with engine.connect() as _:
            pass # Connection successful if this doesn't raise an error
        print("Database connection established successfully.")
        return engine
    except Exception as e:
        print(f"Error connecting to the database: {e}")
        return None

# --- 3. Table Loading Function ---
def load_dataframes_from_db(engine, table_names):
    """
    Loads specified tables from the database into a dictionary of pandas DataFrames,
    checking row counts beforehand.

    Args:
        engine (sqlalchemy.engine.Engine): The SQLAlchemy engine to use for connection.
        table_names (list): A list of table names to load.

    Returns:
        dict: A dictionary where keys are table names (e.g., 'sales')
              and values are the corresponding pandas DataFrames. Returns an empty
              dictionary if errors occur during loading.
    """
    dataframes = {}
    print("\n--- Loading Tables into DataFrames ---")

    with engine.connect() as connection:
        for table in table_names:
            print(f"\nProcessing table: '{table}'")
            try:
                # Step A: Check row count (Good practice for documentation and large datasets)
                count_query = text(f"SELECT COUNT(*) FROM {table};")
                count_result = connection.execute(count_query)
                row_count = count_result.scalar()
                print(f"-> Found {row_count:,} rows.") # Format count with comma

                # Step B: Load table into DataFrame
                print(f"-> Loading '{table}' into DataFrame...")
                df = pd.read_sql_table(table, connection) # Use the existing connection
                dataframes[table] = df # Use table name as key
                print(f"-> Successfully loaded '{table}'. Shape: {df.shape}")

                # Optional Step C: Display memory usage for larger tables if needed
                # if table == 'sales' or table == 'customer':
                #    print("   Memory Usage:")
                #    df.info(memory_usage='deep', verbose=False) # Concise info

            except Exception as e:
                print(f"ERROR loading table '{table}': {e}")
                print(f"Skipping table '{table}'.")
                # Decide if you want to stop entirely or just skip the table
                # If critical tables fail, you might want `return {}` here

    return dataframes

# --- 4. Main Execution ---
print("--- Contoso Data Loading ---")

# Create the database engine
db_engine = create_db_engine(DB_CONFIG)

# Proceed only if the connection was successful
if db_engine:
    # Define the tables we want to load
    tables_to_load = ["currencyexchange", "customer", "product", "sales", "store", "date"]

    # Load the tables into dataframes
    loaded_dfs = load_dataframes_from_db(db_engine, tables_to_load)

    if loaded_dfs:
        print("\n--- Data Loading Summary ---")
        print("Successfully loaded DataFrames for:")
        for name, df in loaded_dfs.items():
            print(f"- {name}: {df.shape}")

        # You can now access your dataframes using the dictionary keys, e.g.:
        # sales_df = loaded_dfs['sales']
        # customer_df = loaded_dfs['customer']
        # product_df = loaded_dfs['product']
        # store_df = loaded_dfs['store']
        # currency_df = loaded_dfs['currencyexchange']
        # date_df = loaded_dfs['date']
        # print("\nExample: First 5 rows of 'customer' data:")
        # if 'customer' in loaded_dfs:
        #     print(loaded_dfs['customer'].head())

        # Reminder about next steps
        print("\nNext Steps: Data cleaning, EDA, and analysis.")
        print("Consider optimizing DataFrame memory usage with .astype() if needed.")

    else:
        print("\nNo DataFrames were loaded due to errors.")

    # Dispose the engine connection pool when completely done
    db_engine.dispose()
    print("\nDatabase connection closed.")

else:
    print("\nScript aborted due to database connection failure.")
    sys.exit(1) # Exit with an error code

--- Contoso Data Loading ---
Database connection established successfully.

--- Loading Tables into DataFrames ---

Processing table: 'currencyexchange'
-> Found 91,325 rows.
-> Loading 'currencyexchange' into DataFrame...
-> Successfully loaded 'currencyexchange'. Shape: (91325, 4)

Processing table: 'customer'
-> Found 104,990 rows.
-> Loading 'customer' into DataFrame...
-> Successfully loaded 'customer'. Shape: (104990, 24)

Processing table: 'product'
-> Found 2,517 rows.
-> Loading 'product' into DataFrame...
-> Successfully loaded 'product'. Shape: (2517, 14)

Processing table: 'sales'
-> Found 199,873 rows.
-> Loading 'sales' into DataFrame...
-> Successfully loaded 'sales'. Shape: (199873, 13)

Processing table: 'store'
-> Found 74 rows.
-> Loading 'store' into DataFrame...
-> Successfully loaded 'store'. Shape: (74, 11)

Processing table: 'date'
-> Found 3,653 rows.
-> Loading 'date' into DataFrame...
-> Successfully loaded 'date'. Shape: (3653, 17)

--- Data Loading Summary 

### Create a master dataframe

In [3]:
# --- Step 1: Set Indices on Dimension Tables (Product, Customer, Store) ---

print("--- Step 1: Setting Indices on Dimension Tables ---")

dimension_tables = ['product', 'customer', 'store']
key_columns = {'product': 'productkey', 'customer': 'customerkey', 'store': 'storekey'}

indices_set = {}

try:
    for table_key in dimension_tables:
        if table_key in loaded_dfs:
            df = loaded_dfs[table_key]
            key_col = key_columns[table_key]

            if key_col in df.columns:
                # Check uniqueness
                if df[key_col].is_unique:
                    df.set_index(key_col, inplace=True)
                    print(f"Index set for '{table_key}_df' using '{key_col}'.")
                    indices_set[table_key] = True
                else:
                    print(f"WARNING: '{key_col}' in {table_key}_df is not unique. Index not set.")
                    indices_set[table_key] = False
            else:
                print(f"WARNING: Key column '{key_col}' not found in {table_key}_df.")
                indices_set[table_key] = False
        else:
            print(f"WARNING: DataFrame '{table_key}' not found in loaded_dfs.")
            indices_set[table_key] = False

except Exception as e:
    print(f"An error occurred during index setting: {e}")


# --- Step 2: Prepare sales_df (Date Conversions) & Create Master Analysis DataFrame ---

print("\n--- Step 2: Preparing sales_df and Creating Master Analysis DataFrame ---")

analysis_df = None # Initialize

if 'sales' in loaded_dfs:
    # Make a copy to avoid modifying the original loaded DataFrame directly
    sales_df = loaded_dfs['sales'].copy()
    print(f"Starting with sales_df (Shape: {sales_df.shape})")

    # Convert date columns to datetime objects
    date_cols = ['orderdate', 'deliverydate']
    print("Converting date columns...")
    for col in date_cols:
        if col in sales_df.columns:
            sales_df[col] = pd.to_datetime(sales_df[col], errors='coerce')
            # Check conversion success
            null_dates = sales_df[col].isnull().sum()
            if null_dates > 0:
                print(f"WARNING: {null_dates} null values found/created in '{col}' after conversion.")
        else:
            print(f"WARNING: Date column '{col}' not found in sales_df.")

    # --- Perform Merges ---
    try:
        analysis_df = sales_df # Start with the prepared sales_df

        # Merge with Product
        prod_key = key_columns['product']
        if indices_set.get('product') and prod_key in analysis_df.columns:
             analysis_df = pd.merge(
                analysis_df, 
                loaded_dfs['product'],
                left_on=prod_key, 
                right_index=True, 
                how='left',
                suffixes=('', '_prod')
            ) # Add suffix if overlapping columns exist
             print(f"Merged with product_df. Shape: {analysis_df.shape}")
        else:
             print(f"Skipping product merge (Index not set or '{prod_key}' missing).")

        # Merge with Customer
        cust_key = key_columns['customer']
        if indices_set.get('customer') and cust_key in analysis_df.columns:
             analysis_df = pd.merge(
                analysis_df, 
                loaded_dfs['customer'],
                left_on=cust_key, 
                right_index=True, 
                how='left',
                suffixes=('', '_cust')
            )
             print(f"Merged with customer_df. Shape: {analysis_df.shape}")
        else:
             print(f"Skipping customer merge (Index not set or '{cust_key}' missing).")

        # Merge with Store
        store_key = key_columns['store']
        if indices_set.get('store') and store_key in analysis_df.columns:
             analysis_df = pd.merge(
                analysis_df, 
                loaded_dfs['store'],
                left_on=store_key, 
                right_index=True, 
                how='left',
                suffixes=('', '_store')
            )
             print(f"Merged with store_df. Shape: {analysis_df.shape}")
        else:
             print(f"Skipping store merge (Index not set or '{store_key}' missing).")

        print("\nVerifying final merged DataFrame ('analysis_df'):")
        print(f"Final Shape: {analysis_df.shape}")
        if analysis_df.shape[0] == loaded_dfs['sales'].shape[0]:
            print("Row count matches original sales data.")
        else:
            print("WARNING: Row count changed after merges. Check merge keys and types.")

        print("\nDataFrame Info:")
        analysis_df.info(memory_usage='deep')

    except Exception as e:
        print(f"An error occurred during merging: {e}")
        analysis_df = None

else:
    print("ERROR: 'sales' DataFrame not found in loaded_dfs.")


# --- Step 3: Set Primary DateTimeIndex ('orderdate') ---

print("\n--- Step 3: Setting DateTimeIndex ---")

primary_date_col = 'orderdate' # Choose the main date column for indexing

if analysis_df is not None and primary_date_col in analysis_df.columns:
    try:
        # Ensure the column is datetime type
        if pd.api.types.is_datetime64_any_dtype(analysis_df[primary_date_col]):
            # Check for NaT values before setting index if critical
            nat_count = analysis_df[primary_date_col].isnull().sum()
            if nat_count > 0:
                 print(f"WARNING: {nat_count} rows have null values in '{primary_date_col}'. They will have NaT index.")
                 # Optionally, handle these rows first (e.g., drop, fill)
                 # analysis_df.dropna(subset=[primary_date_col], inplace=True)

            analysis_df.set_index(primary_date_col, inplace=True)
            # Sort index for time series analysis
            analysis_df.sort_index(inplace=True)
            print(f"Successfully set '{primary_date_col}' column as DateTimeIndex.")
            print(f"Index Type: {type(analysis_df.index)}")
            print(f"Date Range: {analysis_df.index.min()} to {analysis_df.index.max()}")
        else:
            print(f"WARNING: Column '{primary_date_col}' is not datetime type. Cannot set DateTimeIndex.")

    except Exception as e:
        print(f"An error occurred setting DateTimeIndex: {e}")

elif analysis_df is not None:
    print(f"WARNING: Column '{primary_date_col}' not found in analysis_df. Cannot set DateTimeIndex.")
else:
    print("Skipping DateTimeIndex setting as analysis_df was not created.")

# --- Ready for Analysis ---
# Check if analysis_df exists and if its index is a DateTimeIndex type
if analysis_df is not None and ptypes.is_datetime64_any_dtype(analysis_df.index):
    print(f"\nSetup complete! 'analysis_df' is ready with '{primary_date_col}' as DateTimeIndex.")
    # Optional: view first few rows
    # print("\nSample of analysis_df:")
    # print(analysis_df.head())
    # Optional: view columns to check merges and suffixes
    # print("\nColumns in analysis_df:")
    # print(analysis_df.columns)
else:
    # Provide more specific feedback if possible
    if analysis_df is None:
         print("\nSetup incomplete: 'analysis_df' was not created. Please review previous errors.")
    elif not ptypes.is_datetime64_any_dtype(analysis_df.index):
         print(f"\nSetup incomplete: The index of 'analysis_df' is not a DateTimeIndex (Type: {type(analysis_df.index)}). Check Step 3.")
    else:
         print("\nSetup incomplete. Please review warnings/errors above.")

--- Step 1: Setting Indices on Dimension Tables ---
Index set for 'product_df' using 'productkey'.
Index set for 'customer_df' using 'customerkey'.
Index set for 'store_df' using 'storekey'.

--- Step 2: Preparing sales_df and Creating Master Analysis DataFrame ---
Starting with sales_df (Shape: (199873, 13))
Converting date columns...
Merged with product_df. Shape: (199873, 26)
Merged with customer_df. Shape: (199873, 49)
Merged with store_df. Shape: (199873, 59)

Verifying final merged DataFrame ('analysis_df'):
Final Shape: (199873, 59)
Row count matches original sales data.

DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 199873 entries, 0 to 199872
Data columns (total 59 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   orderkey          199873 non-null  int64         
 1   linenumber        199873 non-null  int64         
 2   orderdate         199873 non-null  datetime64[ns]
 3   deliver

### Calculate line totals & currency conversion

In [4]:
analysis_df[['orderkey', 'quantity', 'unitprice', 'unitcost', 'netprice', 'quantity', 'currencycode', 'exchangerate']].sample(5)

,orderkey,quantity,unitprice,unitcost,netprice,quantity,currencycode,exchangerate
orderdate,,,,,,,,
2017-08-30,973004,4,21.0658,10.742,21.0658,4,USD,1.00000
2023-06-18,3091005,1,42.9900,14.240,38.2611,1,EUR,0.91191
2023-06-03,3076045,2,256.0000,117.730,256.0000,2,EUR,0.92911
2019-03-02,1522032,3,492.8000,226.624,438.5920,3,EUR,0.87850
2016-09-26,635004,4,224.9925,114.705,197.9934,4,CAD,1.31637


In [5]:
# --- Pre-Analysis: Calculate Line Totals & Convert Currency ---

print("--- Recalculating Totals and Converting Currency ---")

# Make a copy to ensure we don't modify the original analysis_df accidentally
# if doing interactive work. If this is the main script flow, you can work directly.
df = analysis_df.copy()

# --- 1. Calculate Line Totals (in original currency) ---
try:
    print("Calculating line totals (Revenue, Cost, Profit)...")
    # Revenue = Quantity * Net Price per unit
    df['line_revenue'] = df['quantity'] * df['netprice']

    # Cost = Quantity * Unit Cost per unit
    df['line_cost'] = df['quantity'] * df['unitcost']

    # Profit = Line Revenue - Line Cost
    df['line_profit'] = df['line_revenue'] - df['line_cost']

    print("Line totals calculated.")

except KeyError as e:
    print(f"ERROR: Missing column for calculation: {e}")
    # Handle error - maybe stop processing
    df = None # Indicate failure
except Exception as e:
    print(f"An error occurred during line total calculation: {e}")
    df = None

# --- 2. Currency Conversion to USD ---
if df is not None:
    try:
        print("Converting monetary values to USD...")
        required_cols = ['line_revenue', 'line_cost', 'line_profit', 'currencycode', 'exchangerate']
        if all(col in df.columns for col in required_cols):

            # Handle potential division by zero or null exchange rates
            # Replace 0 or NaN exchange rates with NaN to avoid errors and identify issues
            df['exchangerate'] = df['exchangerate'].replace(0, np.nan)

            # Convert to USD: Divide local currency value by the exchange rate
            # If currency is already USD, the rate should be 1, but division is safe.
            df['revenue_usd'] = df['line_revenue'] / df['exchangerate']
            df['cost_usd'] = df['line_cost'] / df['exchangerate']
            df['profit_usd'] = df['line_profit'] / df['exchangerate']

            print("Currency conversion to USD complete.")

            # Optional: Verify for a non-USD currency
            # print("\nSample conversion check (e.g., first EUR entry):")
            # print(df[df['currencycode'] == 'EUR'][['line_revenue', 'exchangerate', 'revenue_usd']].head(1))

        else:
            print("WARNING: Missing columns required for currency conversion.")
            # Decide how to handle this - maybe stop, or proceed without USD values

    except Exception as e:
        print(f"An error occurred during currency conversion: {e}")
        # Potentially set USD columns to NaN or stop processing
        df = None


# --- 3. Define Core Metrics (Now using USD values) ---
if df is not None:
    print("\n--- Defining Core Metrics (USD) ---")
    revenue_col = 'revenue_usd'
    cost_col = 'cost_usd'
    profit_col = 'profit_usd'
    quantity_col = 'quantity' # Remains the same

    print(f"Using '{revenue_col}' for Revenue analysis.")
    print(f"Using '{cost_col}' for Cost analysis.")
    print(f"Using '{profit_col}' for Profit analysis.")
    print(f"Using '{quantity_col}' for Quantity analysis.")


    # --- 4. Sanity Check Key USD Columns & Index ---
    print(f"\nBasic check on '{revenue_col}':")
    # Check for NaNs introduced by conversion issues
    print(f"Null USD Revenue values: {df[revenue_col].isnull().sum()}")
    print(df[revenue_col].describe())
    print(f"Number of non-positive USD revenue entries: {(df[revenue_col] <= 0).sum()}") # May include nulls

    print(f"\nBasic check on '{quantity_col}':") # Quantity check remains same
    print(df[quantity_col].describe())
    print(f"Number of non-positive quantity entries: {(df[quantity_col] <= 0).sum()}")


    # Check DateTimeIndex (using the processed dataframe)
    print("\nIndex Check:")
    print(f"Index Type: {type(df.index)}")
    if df is not None and ptypes.is_datetime64_any_dtype(df.index):
        print(f"Date Range: {df.index.min()} to {df.index.max()}")
        inferred_freq = pd.infer_freq(df.index)
        print(f"Inferred Index Frequency: {inferred_freq}") # Still likely None
    else:
        print("WARNING: Index is not a DateTimeIndex!")

    # --- Set Plotting Style ---
    # sns.set_theme(style="whitegrid")
    plt.style.use('seaborn-v0_8-whitegrid')
    print("\nPlotting style set.")

    print("\n--- Ready for Analysis ---")

else:
    print("\nProcessing failed. Cannot proceed to analysis.")


# IMPORTANT: Use 'df' for subsequent analysis cells!

# Save master df to a parquet file
df.to_parquet('contoso_100k.parquet')

--- Recalculating Totals and Converting Currency ---
Calculating line totals (Revenue, Cost, Profit)...
Line totals calculated.
Converting monetary values to USD...
Currency conversion to USD complete.

--- Defining Core Metrics (USD) ---
Using 'revenue_usd' for Revenue analysis.
Using 'cost_usd' for Cost analysis.
Using 'profit_usd' for Profit analysis.
Using 'quantity' for Quantity analysis.

Basic check on 'revenue_usd':
Null USD Revenue values: 0
count    199873.000000
mean       1032.021066
std        1986.128568
min           0.601549
25%         105.328080
50%         395.964000
75%        1121.147355
max       57749.932598
Name: revenue_usd, dtype: float64
Number of non-positive USD revenue entries: 0

Basic check on 'quantity':
count    199873.000000
mean          3.143846
std           2.252006
min           1.000000
25%           1.000000
50%           2.000000
75%           4.000000
max          10.000000
Name: quantity, dtype: float64
Number of non-positive quantity entrie

In [6]:
df[['orderkey', 'quantity', 'unitprice', 'unitcost', 'netprice', 'quantity', 'currencycode', 'exchangerate', 'line_revenue', 'line_cost', 'line_profit', 'revenue_usd', 'cost_usd', 'profit_usd']].sample(5)

,orderkey,quantity,unitprice,unitcost,netprice,quantity,currencycode,exchangerate,line_revenue,line_cost,line_profit,revenue_usd,cost_usd,profit_usd
orderdate,,,,,,,,,,,,,,
2023-10-05,3200023,2,22.890,7.580,19.91430,2,USD,1.00000,39.82860,15.160,24.66860,39.828600,15.160000,24.668600
2022-04-23,2670011,3,107.991,55.053,92.87226,3,EUR,0.92447,278.61678,165.159,113.45778,301.380012,178.652633,122.727379
2022-01-05,2562015,1,329.925,168.210,290.33400,1,USD,1.00000,290.33400,168.210,122.12400,290.334000,168.210000,122.124000
2023-10-06,3201026,2,290.000,133.360,258.10000,2,GBP,0.81899,516.20000,266.720,249.48000,630.288526,325.669422,304.619104
2020-11-10,2141003,3,50.400,25.695,50.40000,3,USD,1.00000,151.20000,77.085,74.11500,151.200000,77.085000,74.115000


### Analysis

Theme 1: Sales Performance Over Time (Focus: Time Series Analysis, Aggregation, Visualization)

Q1.1: How did key sales metrics (Total Revenue, Units Sold) trend monthly?

Goal: Understand the overall sales trajectory.
Techniques:
Merge sales with a date dimension table if necessary (or parse DateKey/SalesDate).
Convert date columns to datetime objects (pd.to_datetime).
Set date as index (.set_index()).
Resample data by month (.resample('M')).
Aggregate (.sum(), .count()).
Visualize trends using line plots (matplotlib or seaborn).
Skills Showcased: Time series handling, aggregation, basic plotting.

Q1.2: Is there evidence of seasonality in sales? Which months consistently perform best/worst?

Goal: Identify recurring patterns within the year.
Techniques:
Group aggregated monthly data by month number (.groupby(df.index.month)).
Calculate average monthly performance (.mean()).
Visualize using bar charts or box plots per month (sns.barplot, sns.boxplot).
(Advanced) Use time series decomposition (statsmodels.tsa.seasonal_decompose) to separate trend, seasonality, and residuals.
Skills Showcased: Advanced time series analysis, grouping, statistical visualization, (optional: using stats libraries).
Theme 2: Product Insights (Focus: Merging, Profit Calculation, Ranking, Distribution Analysis)

Q2.1: Which product categories and sub-categories are the most profitable? (Requires product cost info).

Goal: Identify high-margin areas of the business.
Techniques:
Merge sales with product (pd.merge on ProductKey).
Calculate profit per transaction (e.g., (UnitPrice - ProductCost) * Quantity or SalesAmount - (ProductCost * Quantity)). Ensure costs and sales amounts are comparable.
Group by Category, Subcategory (.groupby()).
Aggregate total profit (.sum()).
Visualize using sorted bar charts.
Skills Showcased: Data merging, feature engineering (profit calculation), multi-level grouping, aggregation, comparative visualization.

Q2.2: What does the distribution of sales look like across different price points?

Goal: Understand sales volume relative to product price.
Techniques:
Use the merged sales and product DataFrame.
Create price bins/ranges (pd.cut) if desired.
Analyze the distribution of UnitPrice or SalesAmount using histograms or KDE plots (sns.histplot, sns.kdeplot).
Use scatter plots (sns.scatterplot) to view UnitPrice vs. Quantity sold per transaction.
Skills Showcased: Distribution analysis, visualization (histograms, scatter plots), data binning.
Theme 3: Customer Analysis & Segmentation (Focus: Merging, Feature Engineering, Segmentation Techniques, Statistical Comparison)

Q3.1: What are the demographics (e.g., age group, gender) of our customers and how do they relate to total spending? (Requires customer details like BirthDate, Gender).

Goal: Understand who the customers are and their value.
Techniques:
Merge sales with customer (pd.merge on CustomerKey).
Feature Engineering: Calculate age from BirthDate (using current date pd.Timestamp.now() or a fixed date). Create age groups (pd.cut).
Group by demographic features (.groupby(['AgeGroup', 'Gender'])).
Aggregate total SalesAmount (.sum()) and average SalesAmount (.mean()).
Visualize using bar charts, potentially grouped or stacked.
Skills Showcased: Merging, feature engineering (date calculation, binning), multi-level grouping, comparative visualization.

Q3.2: Can we apply RFM (Recency, Frequency, Monetary) analysis to segment customers?

Goal: Create actionable customer segments based on purchase history.
Techniques:
Requires CustomerKey, SalesDate, SalesAmount.
Calculate Recency (days since last purchase), Frequency (total number of transactions), Monetary (total spend) per customer (.groupby('CustomerKey').agg(...)).
Create RFM scores, often using quantiles (pd.qcut) to create bins (e.g., 1-5 scale).
Combine scores to create RFM segments.
Analyze segment characteristics (size, average R/F/M values).
Skills Showcased: Advanced aggregation, date difference calculation, quantile-based binning (pd.qcut), customer segmentation logic.

Q3.3 (Optional - Hypothesis Test): Is there a statistically significant difference in the average transaction value between male and female customers?

Goal: Apply statistical rigor to an observation.
Techniques:
Use the merged sales and customer DataFrame.
Filter data for male and female customers.
Perform an independent samples t-test (scipy.stats.ttest_ind) on the SalesAmount for the two groups.
Interpret the p-value.
Skills Showcased: Basic hypothesis testing (scipy), data filtering, statistical interpretation.
Theme 4: Store & Geographic Performance (Focus: Merging, Geospatial Ideas - simple, Comparison Visualization)

Q4.1: How do sales compare across different store locations (e.g., cities)?
Goal: Identify top-performing regions.
Techniques:
Merge sales with store (pd.merge on StoreKey).
Group by store attribute like City or Region (.groupby()).
Aggregate total SalesAmount (.sum()).
Visualize using sorted bar charts.
(Optional) If lat/lon available, could do a simple scatter plot on a map background (requires extra libraries like contextily maybe).
Skills Showcased: Merging, grouping, aggregation, visualization.